# 主要内容
6.函数应用和映射
   1. apply():作用于行或列
   2. applymap()：作用于DataFrame的每一个元素
   3. map():作用于Series的每一个元素    
   
7.排序  
   1. sort_index(): 按索引排序
   2. sort_values(): 按值排序
   3. rank()
   
8.有重复label的轴索引

# 6. 函数应用和映射

numpy的ufuncs(element-wise数组方法)也能用在pandas的object上：

In [3]:
import pandas as pd
import numpy as np
frame = pd.DataFrame(np.random.randn(4, 3), columns=list('bde'), 
                     index=['Utah', 'Ohio', 'Texas', 'Oregon'])
frame

,b,d,e
Utah,-0.810048,-0.306993,1.532518
Ohio,1.025970,-0.977624,-0.035398
Texas,0.177357,0.874219,1.011298
Oregon,0.367222,0.812500,0.350723


In [5]:
# 取绝对值
np.abs(frame)

,b,d,e
Utah,0.810048,0.306993,1.532518
Ohio,1.025970,0.977624,0.035398
Texas,0.177357,0.874219,1.011298
Oregon,0.367222,0.812500,0.350723


另一个常用的操作是把一个用在一维数组上的函数，应用在一行或一列上。要用到DataFrame中的apply函数：

# 这里要用到匿名函数，用法如下

In [6]:
f = lambda x: x.max() - x.min()
frame.apply(f)

b    1.836018
d    1.851843
e    1.567915
dtype: float64

这里函数f，计算的是一个series中最大值和最小值的差，在frame中的每一列，这个函数被调用一次。作为结果的series，它的index就是frame的column。

如果传入axis='column'用于apply，那么函数会被用在每一行：

In [8]:
frame.apply(f, axis = 'columns')

Utah      2.342566
Ohio      2.003594
Texas     0.833941
Oregon    0.461777
dtype: float64

像是sum, mean这样的数组统计方法，DataFrame中已经集成了，所以没必要用apply。  
apply不会返回标量，只会返回一个含有多个值的series：

In [9]:
def f(x): 
    return pd.Series([x.min(), x.max()], index=['min', 'max'])

In [10]:
frame.apply(f)

,b,d,e
min,-0.810048,-0.977624,-0.035398
max,1.025970,0.874219,1.532518


element-wise的python函数也能用。假设想要格式化frame中的浮点数，变为string。可以用apply map：

In [12]:
format = lambda x: '%.2f' % x

In [13]:
frame.applymap(format)

,b,d,e
Utah,-0.81,-0.31,1.53
Ohio,1.03,-0.98,-0.04
Texas,0.18,0.87,1.01
Oregon,0.37,0.81,0.35


In [15]:
frame['e'].map(format)

Utah       1.53
Ohio      -0.04
Texas      1.01
Oregon     0.35
Name: e, dtype: object

# 7.排序

按row或column index来排序的话，可以用sort_index方法，会返回一个新的object：

In [16]:
obj = pd.Series(range(4), index=['d', 'a', 'b', 'c'])
obj.sort_index()

a    1
b    2
c    3
d    0
dtype: int32

In [17]:
type(obj)

pandas.core.series.Series

In [18]:
obj

d    0
a    1
b    2
c    3
dtype: int32

在DataFrame，可以用index或其他axis来排序：

In [19]:
frame = pd.DataFrame(np.arange(8).reshape((2, 4)),
                     index=['three', 'one'],
                     columns=['d', 'a', 'b', 'c'])
frame

,d,a,b,c
three,0,1,2,3
one,4,5,6,7


In [20]:
frame.sort_index()

,d,a,b,c
one,4,5,6,7
three,0,1,2,3


In [21]:
frame.sort_index(axis = 1)

,a,b,c,d
three,1,2,3,0
one,5,6,7,4


默认是升序，可以设置降序：

In [22]:
frame.sort_index(axis = 1, ascending = False)

,d,c,b,a
three,0,3,2,1
one,4,7,6,5


通过值来排序，用sort_values方法

In [24]:
frame.sort_values(by = 'b')

,d,a,b,c
three,0,1,2,3
one,4,5,6,7


In [25]:
frame.sort_values(by = ['a', 'b'])

,d,a,b,c
three,0,1,2,3
one,4,5,6,7


In [26]:
obj = pd.Series([7, -5, 7, 4, 2, 0, 4])
obj

0    7
1   -5
2    7
3    4
4    2
5    0
6    4
dtype: int64

In [27]:
obj.sort_values()

1   -5
5    0
4    2
3    4
6    4
0    7
2    7
dtype: int64

ranking（排名）是给有效的数据分配数字。rank方法能用于series和DataFrame，rank方法默认会给每个group一个mean rank（平均排名）。rank 表示在这个数在原来的Series中排第几名，有相同的数，取其排名平均（默认）作为值：

In [28]:
obj.rank()

0    6.5
1    1.0
2    6.5
3    4.5
4    3.0
5    2.0
6    4.5
dtype: float64

比如，4和4的排名是第4名和第五名，取平均得4.5。7和7的排名分别是第六名和第七名，则其排名取平均得6.5。

rank也可以根据数据被观测到的顺序来设定：

In [29]:
obj

0    7
1   -5
2    7
3    4
4    2
5    0
6    4
dtype: int64

In [30]:
obj.rank(method='first')

0    6.0
1    1.0
2    7.0
3    4.0
4    3.0
5    2.0
6    5.0
dtype: float64

这里没有给0和2(指两个数字7)赋予average rank 6.5，而是给第一个看到的7（label 0）设置rank为6，第二个看到的7（label 2）设置rank为7。

也可以设置降序：

In [31]:
obj.rank(ascending=False, method='max')

0    2.0
1    7.0
2    2.0
3    4.0
4    5.0
5    6.0
6    4.0
dtype: float64

In [34]:
obj

0    7
1   -5
2    7
3    4
4    2
5    0
6    4
dtype: int64

dataframe 可以根据行或列来计算rank:

In [35]:
frame = pd.DataFrame({'b': [4.3, 7, -3, 2],
                      'a': [0, 1, 0, 1],
                      'c': [-2, 5, 8, -2.5]})
frame

,a,b,c
0,0,4.3,-2.0
1,1,7.0,5.0
2,0,-3.0,8.0
3,1,2.0,-2.5


In [36]:
frame.rank(axis='columns') # columns表示列与列之间的排序（即每一行里数据间的排序）

,a,b,c
0,2.0,3.0,1.0
1,1.0,3.0,2.0
2,2.0,1.0,3.0
3,2.0,3.0,1.0


# 8.有重复label的轴索引

当有重复索引时

In [37]:
obj = pd.Series(range(5), index=['a', 'a', 'b', 'b', 'c'])
obj

a    0
a    1
b    2
b    3
c    4
dtype: int32

index的is_unique能告诉我们label是否是唯一的：

In [39]:
obj.index.is_unique

False

In [40]:
obj['a']

a    0
a    1
dtype: int32

In [41]:
obj['c']

4

In [42]:
df = pd.DataFrame(np.random.randn(4, 3), index=['a', 'a', 'b', 'b'])
df

,0,1,2
a,-2.043309,1.433332,-1.963896
a,0.532380,1.188984,0.287367
b,-2.325143,0.040920,0.400178
b,-0.500287,-0.001362,0.274431


In [44]:
df.loc['a']

,0,1,2
a,-2.043309,1.433332,-1.963896
a,0.532380,1.188984,0.287367
